#### Algorithm.
Unique grid patterns by group actions. Use [Polya's Enumeration Theorem](https://en.wikipedia.org/wiki/P%C3%B3lya_enumeration_theorem). 



In [1]:
""" Challenge 5.0. Get unique grid configuarations by group """

import collections as col, itertools as it, operator as opt, functools as ft

## Return products of all numbers in a list- analogous to sum...
prod = lambda lst: ft.reduce(opt.mul, lst)

## Generates factorials for a range [0, N]...
def GetFactorials(N):
    Factorials = col.defaultdict(None, {0: 1, 1: 1})
    for n in range(2, N + 1):
        Factorials[n] = Factorials[n - 1] * n
    return Factorials

## Generates GCDs for a grid of size N1 x N2 (only unique combinations)...
def GetGCDs(N1, N2):
    ## Since we're generating GCD values only for the unique 
    ## combinations, #rows <= #columns
    m, n = (N1 + 1, N2 + 1) if (N1 <= N2) else (N2 + 1, N1 + 1)
    
    GCD = col.defaultdict(None, {(1, 1): 1})
    for x in range(1, m):
        for y in range(x, n):
            if x == y:
                GCD[(x, y)] = x
            elif 2*x <= y:         # Only pairs (x, y) where x <= y are valid
                GCD[(x, y)] = GCD[(x, y - x)]
            else:
                GCD[(x, y)] = GCD[(y - x, x)]
    return GCD

## Generate ascending compositions (ordered partitions) of an integer n...
def PartitionGenerator(n):
    Partition = [0 for i in range(n + 1)]
    k = 1      # Index for parition length
    Partition[1] = n
    while k != 0:
        ## Start with 1...
        x = Partition[k - 1] + 1
        y = Partition[k] - 1
        k -= 1
        
        ## Caclulate inner partitions...
        while x <= y:
            Partition[k] = x
            y -= x
            k += 1
        
        Partition[k] = x + y
        
        ## Return i-th partition...
        yield Partition[:k + 1]


def solution(w, h, s):
    ## Build tables for later access...
    FactorialTable, GCDtable = GetFactorials(max(w, h)), GetGCDs(w, h)
    
    ## Necessary functions...
    factorial = lambda n: FactorialTable[n]
    gcd = lambda x, y: GCDtable[(x, y)] if (x <= y) else GCDtable[(y, x)]
    # coefficient = lambda p: prod([q**k * factorial(k) for q, k in col.Counter(p).items()])
    coefficient = lambda p: factorial(sum(p)) / prod([q**k * factorial(k) for q, k in col.Counter(p).items()])
    
    ## Count configurations [implementing Polya's Enumeration Theorem]...
    grids = 0
    for p_w in PartitionGenerator(w):
        c_w = coefficient(p_w)
        for p_h in PartitionGenerator(h):
            c_h = coefficient(p_h)
            # grids += s**sum([gcd(a, b) for a, b in it.product(p_w, p_h)]) / (c_w * c_h)
            grids += c_w * c_h * s**sum([gcd(a, b) for a, b in it.product(p_w, p_h)])
    # grids = str(int(grids))
    grids = str(int( grids / (factorial(w) * factorial(h)) ))
    return grids


In [2]:
## Examples...
%time print("Configuration count = ", solution(w = 2, h = 2, s = 2))
%time print("Configuration count = ", solution(w = 2, h = 3, s = 4))


Configuration count =  7
Wall time: 997 µs
Configuration count =  430
Wall time: 998 µs
